In [1]:
import requests
import json
import pandas as pd
from pprint import pprint

In [11]:
from config import conn_str  
from config import api_key
from dboperations import * 

In [3]:
conn = connect2db(conn_str)

Connection Succesful


In [4]:
# set aviationstack.com api request parameters
params = {'access_key': api_key,
          'arr_iata': 'LAX' }

api_link = requests.get('https://api.aviationstack.com/v1/flights', params) 

In [5]:
# get api response
response = api_link.json()
# pprint(response)

In [6]:
# parse data
airline = []
flight = []
departure = []
arrival = []

airline.append(response['data'][0]['airline']) # index 
# print(response['data'][0]['airline'])  
flight.append(response['data'][0]['flight'])
departure.append(response['data'][0]['departure'])
arrival.append(response['data'][0]['arrival'])

In [7]:
print(f"flight: {flight}")

flight: [{'number': '447', 'iata': 'CZ447', 'icao': 'CSN447', 'codeshared': None}]


In [8]:
# create dataframes
airlines_df = pd.DataFrame(airline)
flight_df = pd.DataFrame(flight)
departure_df = pd.DataFrame(departure)
arrival = pd.DataFrame(arrival)

In [9]:
# clean airline frame for table
airlines_df.reset_index(inplace = True) #need that for following loop 
airlines_df = airlines_df.rename(columns = {"index": "airline_id", "name": "airline"})  
airlines_df

,airline_id,airline,iata,icao
0,0,China Southern Airlines,CZ,CSN


In [10]:
# airline insert df to table
insertvalues(conn, "airlines", airlines_df)

Values Succesfully Insert To airlines


In [12]:
flight_df

,number,iata,icao,codeshared
0,447,CZ447,CSN447,None


In [ ]:
# combine flight&departure&arrival
merge_df = pd.merge(info_df, items_df, 
merge_df



In [ ]:
closeconnection(conn);